# EITC -> CTC

Replace MD EITC with a filer credit + child credit such that there are no losers (except adult dependents). Vary this 3x3:
1. Child credit
1. Filer credit for singles
1. Filer credit for couples

for each:
1. Amount
2. Phase-out start
3. Phase-out rate

Compute the filer credit amounts from the other 7 parameters.

In [1]:
import numpy as np
import pandas as pd
from openfisca_us import Microsimulation, IndividualSim


In [2]:
from typing import Tuple
from openfisca_us import IndividualSim
from openfisca_core.model_api import *


def modifier_abolish_eitc(parameters):
    eitc = parameters.gov.states.md.tax.income.credits.eitc
    eitc.childless.percent_match.update(period="year:2020:10", value=0)
    eitc.refundable_match.update(period="year:2020:10", value=0)
    eitc.non_refundable_match.update(period="year:2020:10", value=0)
    return parameters

def modifier_eitc_2023(parameters):
    # The refundable EITC match will fall from 45% to 28% in 2023.
    eitc = parameters.gov.states.md.tax.income.credits.eitc
    eitc.refundable_match.update(period="year:2020:10", value=0.28)
    return parameters


class abolish_eitc(Reform):
    def apply(self):
        self.modify_parameters(modifier_abolish_eitc)

class eitc_2023(Reform):
    def apply(self):
        self.modify_parameters(modifier_eitc_2023)


def make_sim(adults, children, reform=None):
    if reform is None:
        sim = IndividualSim(year=2022)
    else:
        sim = IndividualSim(reform, year=2022)
    sim.add_person(name="head", age=25)
    members = ["head"]
    if adults == 2:
        sim.add_person(name="spouse", age=25)
        members += ["spouse"]
    for i in range(children):
        child = "child{}".format(i)
        sim.add_person(name=child, age=6)
        members += [child]
    sim.add_tax_unit(name="tax_unit", members=members, premium_tax_credit=0)
    sim.add_household(name="household", members=members, state_code="MD")
    sim.vary("employment_income", max=100_000, step=100)
    return sim



earnings_by_household = {}
gains_by_household = {}
gains_by_household_2023 = {}
for adults in [1, 2]:
    for children in [0, 1, 2, 3]:
        baseline_sim = make_sim(adults, children)
        eitc_2023_sim = make_sim(adults, children, eitc_2023)
        eitc_abolish_sim = make_sim(adults, children, abolish_eitc)
        earnings_by_household[(adults, children)] = baseline_sim.calc(
            "adjusted_gross_income"
        )[0]
        gains_by_household[(adults, children)] = (
            eitc_abolish_sim.calc("spm_unit_net_income")[0]
            - baseline_sim.calc("spm_unit_net_income")[0]
        )
        gains_by_household_2023[(adults, children)] = (
            eitc_abolish_sim.calc("spm_unit_net_income")[0]
            - eitc_2023_sim.calc("spm_unit_net_income")[0]
        )


In [14]:
make_sim(2, 0).calc("filing_status")

EnumArray([[<FilingStatus.JOINT: 'Joint'> <FilingStatus.JOINT: 'Joint'>
  <FilingStatus.JOINT: 'Joint'> ... <FilingStatus.JOINT: 'Joint'>
  <FilingStatus.JOINT: 'Joint'> <FilingStatus.JOINT: 'Joint'>]])

In [8]:
LOSS_ALLOWANCE = 0

import plotly.express as px


def get_filer_credits(
    ctc_amount: float,
    ctc_phase_out_threshold: float,
    ctc_phase_out_rate: float,
    filer_single_phase_out_threshold: float,
    filer_couple_phase_out_threshold: float,
    filer_single_phase_out_rate: float,
    filer_couple_phase_out_rate: float,
    gains_by_household: dict,
) -> Tuple[float, float]:

    # First, find single filer credit amounts

    single_filer_credits = []

    for children in [0, 1, 2, 3]:
        earnings = earnings_by_household[(1, children)]
        gains = gains_by_household[(1, children)]
        income_over_ctc_threshold = max_(0, earnings - ctc_phase_out_threshold)
        ctc_phased_out_amount = income_over_ctc_threshold * ctc_phase_out_rate
        ctc_value = max_(0, ctc_amount * children - ctc_phased_out_amount)
        gains = gains_by_household[(1, children)] + ctc_value
        # Find credit amount satisfying earnings < phase-out threshold
        credit_amount_pre_threshold = max(
            -gains * (earnings < filer_single_phase_out_threshold)
        )
        # Find credit amount satisfying earnings >= phase-out threshold
        amount_phased_out = (
            max_(0, earnings - filer_single_phase_out_threshold)
            * filer_single_phase_out_rate
        )
        credit_amount_post_threshold = max(
            (-gains + amount_phased_out)
            * (earnings >= filer_single_phase_out_threshold)
        )

        credit_amount = max(
            credit_amount_pre_threshold, credit_amount_post_threshold
        )
        single_filer_credits.append(credit_amount)

    single_filer_credit = max(single_filer_credits)

    # Then, find couple filer credit amounts

    couple_filer_credits = []

    for children in [0, 1, 2, 3]:
        earnings = earnings_by_household[(2, children)]
        income_over_ctc_threshold = max_(0, earnings - ctc_phase_out_threshold)
        ctc_phased_out_amount = income_over_ctc_threshold * ctc_phase_out_rate
        ctc_value = max_(0, ctc_amount * children - ctc_phased_out_amount)
        gains = gains_by_household[(1, children)] + ctc_value
        # Find credit amount satisfying earnings < phase-out threshold
        credit_amount_pre_threshold = max(
            -gains * (earnings < filer_couple_phase_out_threshold)
        )
        # Find credit amount satisfying earnings >= phase-out threshold
        amount_phased_out = (
            earnings - filer_couple_phase_out_threshold
        ) * filer_couple_phase_out_rate
        credit_amount_post_threshold = max(-gains + amount_phased_out)
        credit_amount = max(
            credit_amount_pre_threshold, credit_amount_post_threshold
        )
        couple_filer_credits.append(credit_amount)

    couple_filer_credit = max(couple_filer_credits)

    return (
        single_filer_credit - LOSS_ALLOWANCE,
        couple_filer_credit - LOSS_ALLOWANCE,
    )


In [15]:
# Test filer credit.
# Should be $530 for single filer and $250 for couple filer.
get_filer_credits(
    ctc_amount=1e6,
    ctc_phase_out_threshold=0,
    ctc_phase_out_rate=0,
    filer_single_phase_out_threshold=0,
    filer_couple_phase_out_threshold=0,
    filer_single_phase_out_rate=0,
    filer_couple_phase_out_rate=0,
    gains_by_household=gains_by_household
)


(530.0, 530.0)

In [22]:
from openfisca_us import Microsimulation

sim = Microsimulation()

filing_status = sim.calc("filing_status").values
is_couple_filer = filing_status == "JOINT"
is_single_filer = ~is_couple_filer
agi = sim.calc("adjusted_gross_income").values
child_count = sim.calc("is_child", map_to="tax_unit").values
tax_unit_weight = sim.calc("tax_unit_weight").values
in_md = sim.simulation.tax_unit.household("state_code_str", 2022) == "MD"
tax_unit_weight *= in_md


In [23]:
from scipy.optimize import differential_evolution


def budgetary_impact(
    ctc_amount: float,
    ctc_phase_out_threshold: float,
    ctc_phase_out_rate: float,
    filer_single_phase_out_threshold: float,
    filer_couple_phase_out_threshold: float,
    filer_single_phase_out_rate: float,
    filer_couple_phase_out_rate: float,
    gains_by_household: dict,
) -> float:
    single_filer_credit, couple_filer_credit = get_filer_credits(
        ctc_amount,
        ctc_phase_out_threshold,
        ctc_phase_out_rate,
        filer_single_phase_out_threshold,
        filer_couple_phase_out_threshold,
        filer_single_phase_out_rate,
        filer_couple_phase_out_rate,
        gains_by_household
    )
    ctc_income_over_threshold = max_(0, agi - ctc_phase_out_threshold)
    ctc_amount_after_phase_out = max_(
        0,
        ctc_amount * child_count
        - ctc_income_over_threshold * ctc_phase_out_rate,
    )
    ctc_cost = (ctc_amount_after_phase_out * tax_unit_weight).sum()

    threshold = where(
        is_single_filer,
        filer_single_phase_out_threshold,
        filer_couple_phase_out_threshold,
    )
    rate = where(
        is_single_filer,
        filer_single_phase_out_rate,
        filer_couple_phase_out_rate,
    )
    filer_income_over_threshold = max_(0, agi - threshold)
    filer_amount = where(
        is_single_filer, single_filer_credit, couple_filer_credit
    )
    filer_amount_after_phase_out = max_(
        0, filer_amount - filer_income_over_threshold * rate
    )
    filer_cost = (filer_amount_after_phase_out * tax_unit_weight).sum()

    return ctc_cost + filer_cost


In [24]:
from scipy.optimize import differential_evolution

res = differential_evolution(
    lambda x: budgetary_impact(*x, gains_by_household=gains_by_household),
    bounds=[
        (0, 10_000),
        (0, 40_000),
        (0, 1),
        (0, 40_000),
        (0, 80_000),
        (0, 1),
        (0, 1),
    ],
    maxiter=1_000,
)


In [27]:
print(
    f"CTC amount: ${res.x[0]:,.0f}\nCTC phase-out threshold: ${res.x[1]:,.0f}\nCTC phase-out rate: {res.x[2]:.2%}\nSingle filer phase-out threshold: ${res.x[3]:,.0f}\nCouple filer phase-out threshold: ${res.x[4]:,.0f}\nSingle filer phase-out rate: {res.x[5]:.2%}\nCouple filer phase-out rate: {res.x[6]:.2%}"
)
single_filer_credit, couple_filer_credit = get_filer_credits(*res.x, gains_by_household=gains_by_household)
print(
    f"Single filer credit: ${single_filer_credit:,.0f}\nCouple filer credit: ${couple_filer_credit:,.0f}"
)
print(f"Budgetary impact: ${budgetary_impact(*res.x, gains_by_household=gains_by_household)/1e9:,.1f}bn")


CTC amount: $1,211
CTC phase-out threshold: $30,795
CTC phase-out rate: 14.75%
Single filer phase-out threshold: $9,558
Couple filer phase-out threshold: $14,140
Single filer phase-out rate: 0.59%
Couple filer phase-out rate: 0.63%
Single filer credit: $530
Couple filer credit: $538
Budgetary impact: $1.1bn


In [29]:
from scipy.optimize import differential_evolution

res_2023 = differential_evolution(
    lambda x: budgetary_impact(*x, gains_by_household=gains_by_household_2023),
    bounds=[
        (0, 10_000),
        (0, 40_000),
        (0, 1),
        (0, 40_000),
        (0, 80_000),
        (0, 1),
        (0, 1),
    ],
    maxiter=1_000,
)


In [30]:
print(
    f"CTC amount: ${res_2023.x[0]:,.0f}\nCTC phase-out threshold: ${res_2023.x[1]:,.0f}\nCTC phase-out rate: {res_2023.x[2]:.2%}\nSingle filer phase-out threshold: ${res_2023.x[3]:,.0f}\nCouple filer phase-out threshold: ${res_2023.x[4]:,.0f}\nSingle filer phase-out rate: {res_2023.x[5]:.2%}\nCouple filer phase-out rate: {res_2023.x[6]:.2%}"
)
single_filer_credit, couple_filer_credit = get_filer_credits(*res.x, gains_by_household=gains_by_household_2023)
print(
    f"Single filer credit: ${single_filer_credit:,.0f}\nCouple filer credit: ${couple_filer_credit:,.0f}"
)
print(f"Budgetary impact: ${budgetary_impact(*res_2023.x, gains_by_household=gains_by_household_2023)/1e9:,.1f}bn")


CTC amount: $576
CTC phase-out threshold: $34,614
CTC phase-out rate: 9.55%
Single filer phase-out threshold: $10,583
Couple filer phase-out threshold: $11,126
Single filer phase-out rate: 0.59%
Couple filer phase-out rate: 0.60%
Single filer credit: $530
Couple filer credit: $538
Budgetary impact: $1.0bn


In [ ]:
# 1. Find credit amount satisfying
# For each sim:
# We have Gain = [-3, -4, -5, ...]
# And Earnings = [0, 1, 2, ...]
# Threshold
# And Rate
# Phased out amount =
# X = Rate * Earnings - Gain

# Gain = [-50, -70, -10, 10]
# Earnings = [0, 100, 200, 300]
# Threshold =

# % phased out = (earnings - threshold) * rate / FilerCredit
# X * rate = threshold * rate + % phased out
# X = (threshold * rate + % phased out) / rate

# GainToOffset = Gain / % phased out
# FilerCredit = max(GainToOffset)

# Example:
# Loss = 50
# Earnings = 10000
# Threshold = 5000
# Rate = 10%
# Amount phased out = 10% * (10000 - 5000) = 500
# % phased out = 500 / FilerCredit
# FilerCredit = Loss / (1 - % phased out) = 50 / (1 - (500 / FilerCredit))
# FilerCredit = 550
# % phased out = 500 / 550 = 0.91
